### Formulación 1:
2 Clases:
* Falla
* No falla

### Formulación 2:
2 Clases:
* Falla (con alcance definido)
* No falla

### Formulación 3:
3 Clases:
* Falla zona primaria
* Falla zona secundaria
* No Falla

In [2]:
# Para obtener los resultados del modelo
# from utils.detector import validate_csv, load_events
from utils.detector import validate_csv, load_events

# Para crear matrices de confusión y trabajar con DataFrames
import pandas as pd
import numpy as np

# Para leer las carpetas con los CSV
import os

# Para utilizar Expresiones regulares
import re

In [3]:
# Crear las etiquetas 
zone_01 = "806"
zone_02 = "112"
zone_10 = "828"
zone_09 = "124"
zone_11 = "224"
zone_15 = "860"
zone_16 = "840"
zone_dict = {
    "R01": [zone_01, zone_02],
    "R10": [zone_10, zone_09],
    "R15": [zone_15, zone_16],
}

#### Formulación 1

In [6]:
# Dirección de carpeta con los CSV de fallas

# Carpeta con CSV de prueba
# PATH = "C:\\Users\\aherrada\\OneDrive - Universidad del Norte\\Uninorte\\DetectionDataBase\\PruebaValidacion"

# Carpeta con CSV Formulación 1
PATH = "C:\\Users\\aherrada\\OneDrive - Universidad del Norte\\Uninorte\\DetectionDataBase\\Formulacion1_csv"

# Carpeta con Dataset correcto
# PATH = "C:\\Users\\aherrada\\OneDrive - Universidad del Norte\\Uninorte\\DetectionDataBase\\septDataBaseCSV\\Fallas"

casos = os.listdir(PATH)
predicted_condition = {}
actual_condition = {}
case_dict = {}
relay_list = ["R01", "R10", "R15"]
# relay_list = ["R01"]
signal_list = ["Ia", "Ib", "Ic"]
pattern = re.compile("(No)*(Fault)")
pattern2 = re.compile("B(\d{3})")

for i, caso in enumerate(casos):
    print(f"Sistema {caso}")
    PATH_caso = f"{PATH}\\{caso}"
    m = pattern.search(caso)
    m2 = pattern2.search(caso)
    
    labels = []
    results = []
    # break_flag = 0
    for relay in relay_list:
        # if (m[0] == "Fault") and (m2[1] not in zone_dict[relay]): 
        #     break_flag = 1
        #     break
        alarm = 0
        op_window = 137
        if m[0] == "Fault":
            correct_label = 1
        elif m[0] == "NoFault":
            correct_label = 0
        labels.append(correct_label)
        for phase in signal_list:
            signal_name = f"{relay}{phase}"
            trip, t = validate_csv(PATH_caso, signal_name)
            if t < op_window:
                op_window = t
                alarm = trip
        result = alarm
        results.append(result)
    # if break_flag == 1:
    #     continue
    case_dict[caso] = results
    actual_condition[caso] = labels
print(case_dict)
event_matrix = pd.DataFrame(case_dict)
event_matrix.index = relay_list
event_matrix = event_matrix.transpose()
# print(event_matrix)
print(event_matrix)

Sistema Fault01_B112_RF1e-05.csv
Sistema Fault01_B112_RF40.csv
Sistema Fault01_B112_RF80.csv
Sistema Fault01_B124_RF1e-05.csv
Sistema Fault01_B124_RF40.csv
Sistema Fault01_B124_RF80.csv
Sistema Fault01_B224_RF1e-05.csv
Sistema Fault01_B224_RF40.csv
Sistema Fault01_B224_RF80.csv
Sistema Fault01_B806_RF1e-05.csv
Sistema Fault01_B806_RF40.csv
Sistema Fault01_B806_RF80.csv
Sistema Fault01_B828_RF1e-05.csv
Sistema Fault01_B828_RF40.csv
Sistema Fault01_B828_RF80.csv
Sistema Fault01_B840_RF1e-05.csv
Sistema Fault01_B840_RF40.csv
Sistema Fault01_B840_RF80.csv
Sistema Fault01_B860_RF1e-05.csv
Sistema Fault01_B860_RF40.csv
Sistema Fault01_B860_RF80.csv
Sistema Fault02_B112_RF1e-05.csv
Sistema Fault02_B112_RF40.csv
Sistema Fault02_B112_RF80.csv
Sistema Fault02_B124_RF1e-05.csv
Sistema Fault02_B124_RF40.csv
Sistema Fault02_B124_RF80.csv
Sistema Fault02_B224_RF1e-05.csv
Sistema Fault02_B224_RF40.csv
Sistema Fault02_B224_RF80.csv
Sistema Fault02_B806_RF1e-05.csv
Sistema Fault02_B806_RF40.csv
Sistema

In [7]:

# print(actual_condition)
def unpack_results(label_index: int) -> np.matrix:
    confussion_matrix = np.matrix('0 0; 0 0')
    for trips, labels in zip(case_dict.values(), actual_condition.values()):
        label = labels[label_index]
        trip = trips[label_index]
        if label == 0:
            if trip == 1:
                confussion_matrix[1,0] += 1
            if trip == 0:
                confussion_matrix[1,1] += 1
        if label == 1:
            if trip == 1:
                confussion_matrix[0,0] += 1
            if trip == 0:
                confussion_matrix[0,1] += 1
    
    confussion_matrix = pd.DataFrame(
        data = confussion_matrix,
        index = ["Actual positives", "Actual Negatives"],
        columns = ["Pred. Positives", "Pred. Negatives"]
    )
    return confussion_matrix


confussion_matrix = unpack_results(0)
print(f"Matriz de confusión R01:\n\n {confussion_matrix}\n\n")
confussion_matrix = unpack_results(1)
print(f"Matriz de confusión R10:\n\n {confussion_matrix}\n\n")
confussion_matrix = unpack_results(2)
print(f"Matriz de confusión R15:\n\n {confussion_matrix}\n\n")



Matriz de confusión R01:

                   Pred. Positives  Pred. Negatives
Actual positives              231                0
Actual Negatives                0               14


Matriz de confusión R10:

                   Pred. Positives  Pred. Negatives
Actual positives              221               10
Actual Negatives                0               14


Matriz de confusión R15:

                   Pred. Positives  Pred. Negatives
Actual positives              149               82
Actual Negatives                2               12




#### Formulación 2

In [233]:
import random
# Dirección de carpeta con los CSV de fallas

# Carpeta con CSV de prueba
# PATH = "C:\\Users\\aherrada\\OneDrive - Universidad del Norte\\Uninorte\\DetectionDataBase\\PruebaValidacion"

# Carpeta con Dataset correcto
PATH = "C:\\Users\\aherrada\\OneDrive - Universidad del Norte\\Uninorte\\DetectionDataBase\\septDataBaseCSV\\Fallas"

casos = os.listdir(PATH)
predicted_condition = {}
actual_condition = {}
case_dict = {}
# relay_list = ["R01", "R10", "R15"]
relay_list = ["R01"]
signal_list = ["Ia", "Ib", "Ic"]
pattern = re.compile("B(\d{3})")



for i, caso in enumerate(casos):
    print(f"Sistema {caso}")
    PATH_caso = f"{PATH}\\{caso}"
    m = pattern.findall(PATH_caso)
    labels = []
    results = []
    for relay in relay_list:
        alarm = 0
        op_window = 137
        rand_num  = 0
        if m[0] in zone_dict[relay]:
            correct_label = 1
        else:
            rand_num = random.randint(0,9)
            if rand_num > 3:
                continue

            correct_label = 0
        labels.append(correct_label)
        for phase in signal_list:
            signal_name = f"{relay}{phase}"
            trip, t = validate_csv(PATH_caso, signal_name)
            if t < op_window:
                op_window = t
                alarm = trip
        result = alarm
        results.append(result)
    if rand_num < 4:
        case_dict[caso] = results
        actual_condition[caso] = labels

# case_dict = {case: cases for case, cases in case_dict.items() if len(cases) >=2}

event_matrix = pd.DataFrame(case_dict)
event_matrix.index = relay_list
event_matrix = event_matrix.transpose()
# print(event_matrix)

Sistema Fault01_B112_RF1e-05.csv
Sistema Fault01_B112_RF40.csv
Sistema Fault01_B112_RF80.csv
Sistema Fault01_B124_RF1e-05.csv
Sistema Fault01_B124_RF40.csv
Sistema Fault01_B124_RF80.csv
Sistema Fault01_B224_RF1e-05.csv
Sistema Fault01_B224_RF40.csv
Sistema Fault01_B224_RF80.csv
Sistema Fault01_B806_RF1e-05.csv
Sistema Fault01_B806_RF40.csv
Sistema Fault01_B806_RF80.csv
Sistema Fault01_B828_RF1e-05.csv
Sistema Fault01_B828_RF40.csv
Sistema Fault01_B828_RF80.csv
Sistema Fault01_B840_RF1e-05.csv
Sistema Fault01_B840_RF40.csv
Sistema Fault01_B840_RF80.csv
Sistema Fault01_B860_RF1e-05.csv
Sistema Fault01_B860_RF40.csv
Sistema Fault01_B860_RF80.csv
Sistema Fault02_B112_RF1e-05.csv
Sistema Fault02_B112_RF40.csv
Sistema Fault02_B112_RF80.csv
Sistema Fault02_B124_RF1e-05.csv
Sistema Fault02_B124_RF40.csv
Sistema Fault02_B124_RF80.csv
Sistema Fault02_B224_RF1e-05.csv
Sistema Fault02_B224_RF40.csv
Sistema Fault02_B224_RF80.csv
Sistema Fault02_B806_RF1e-05.csv
Sistema Fault02_B806_RF40.csv
Sistema

In [234]:
import random
def formulacion2(relay_name):
    # Carpeta con CSV de prueba
    # PATH = "C:\\Users\\aherrada\\OneDrive - Universidad del Norte\\Uninorte\\DetectionDataBase\\PruebaValidacion"

    # Carpeta con Dataset correcto
    PATH = "C:\\Users\\aherrada\\OneDrive - Universidad del Norte\\Uninorte\\DetectionDataBase\\septDataBaseCSV\\Fallas"

    # Carpeta con CSV Formulación 1
    # PATH = "C:\\Users\\aherrada\\OneDrive - Universidad del Norte\\Uninorte\\DetectionDataBase\\Formulacion1_csv"   
    casos = os.listdir(PATH)
    predicted_condition = {}
    actual_condition = {}
    case_dict = {}
    # relay_list = ["R01", "R10", "R15"]
    relay_list = [relay_name]
    signal_list = ["Ia", "Ib", "Ic"]
    pattern = re.compile("B(\d{3})")

    for i, caso in enumerate(casos):
        # print(f"Sistema {caso}")
        PATH_caso = f"{PATH}\\{caso}"
        m = pattern.findall(PATH_caso)
        labels = []
        results = []
        for relay in relay_list:
            alarm = 0
            op_window = 137
            rand_num  = 0
            if m[0] in zone_dict[relay]:
                correct_label = 1
            else:
                rand_num = random.randint(0,9)
                if rand_num > 5:
                    continue

                correct_label = 0
            labels.append(correct_label)
            for phase in signal_list:
                signal_name = f"{relay}{phase}"
                trip, t = validate_csv(PATH_caso, signal_name)
                if t < op_window:
                    op_window = t
                    alarm = trip
            result = alarm
            results.append(result)
        if rand_num < 6:
            case_dict[caso] = results
            actual_condition[caso] = labels

    event_matrix = pd.DataFrame(case_dict)
    event_matrix.index = relay_list
    event_matrix = event_matrix.transpose()
    return event_matrix

confussion_matrix = formulacion2("R01")
confussion_matrix = unpack_results(0)
print(f"Matriz de confusión R01:\n\n {confussion_matrix}\n\n")
confussion_matrix = formulacion2("R10")
confussion_matrix = unpack_results(0)
print(f"Matriz de confusión R01:\n\n {confussion_matrix}\n\n")
confussion_matrix = formulacion2("R15")
confussion_matrix = unpack_results(0)
print(f"Matriz de confusión R01:\n\n {confussion_matrix}\n\n")

Matriz de confusión R01:

                   Pred. Positives  Pred. Negatives
Actual positives               66                0
Actual Negatives               57                0


Matriz de confusión R01:

                   Pred. Positives  Pred. Negatives
Actual positives               66                0
Actual Negatives               57                0


Matriz de confusión R01:

                   Pred. Positives  Pred. Negatives
Actual positives               66                0
Actual Negatives               57                0


